## Analysis GASS

This notebook is meant to analyze the results of the GASS optimisation testing.



**History:** <br>
01.05.2018: moving to python3 <br>
02.05.2018: test sync dropbox/fithub/kubox

In [1]:
import matplotlib.pyplot as pl
import numpy as np
import pickle

%matplotlib inline

dirResults = "./results/"

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
def read_pickle(filedata):
    
    with open(dirResults+filedata, 'rb') as f:
        data = pickle.load(f,encoding='latin1')
    
    return(data)

##########################################

def find_range_value(arr,val):
    "find start and end for val in arr"
    
    start, end = 0 , 0
    startFound = False
    
    for i in range(len(arr)):
        if arr[i] == val and not startFound:
            start = i
            startFound = True
            
        if arr[i] == val and startFound: 
            end = i
            
    return(start, end)
        
#########################################       
    
def stat_data(key,data):
    "Compute the stats on the Score"
    
    runningValue = data[key][0]
    endInd = len(data[key])
    
    tempDat = []
    tempScoMean = []
    tempScoStd  = []
    
    endFound = False
    while(not endFound):
        ist , ien = find_range_value(data[key], runningValue)
        sco = np.array(data['Score'][ist:ien])
        
        mn = np.mean(sco)
        std =  np.std(sco)
        
        tempDat.append(runningValue)
        tempScoMean.append(mn)
        tempScoStd.append(std)
            
        if ien < endInd-1:
            runningValue = runningValue = data[key][ien+1]
        else:
            endFound = True
            
    dataStat = {}

    dataStat[key] = tempDat
    
    dataStat['Score'] = {}
    dataStat['Score']['Mean'] = tempScoMean
    dataStat['Score']['Std'] = tempScoStd
        
    return(dataStat) 
  
    #####################################################################################################

### Iteration -------

This plot shows the impact of the iteration number on the score with a dispersion on 10 values for each point. <br>

**Results** : 150 is a conservative value for convergence. We will use it in the following simulation

Last simulation:  Results_30-04-2018_15:57:52


In [ ]:
data = read_pickle('score_iteration.pkl')

d = stat_data('Iteration',data)

# print(d['Score']['Mean'])


In [ ]:
## Plot
xx = d['Iteration']
yy = d['Score']['Mean']
err = d['Score']['Std']

pl.figure(figsize=(10,6))

pl.errorbar(xx , yy , err)

ax = pl.gca()
ax.set_xlim(0., 250)
ax.set_ylim(-0.3, 0.1)

pl.xlabel("Iteration")
pl.ylabel("Score")
pl.show()

### Population size -------

The plot shows the correlation between the Score and the population size. <br>



In [ ]:
# data = read_pickle('score_population.pkl')

# d = stat_data('Elitism',data)

# print(d['Score']['Mean'])